##**GPU verification**

In [ ]:
!nvidia-smi

Mon Feb 16 15:29:48 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8             10W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

##**Installing deployment libraries**

In [ ]:
!pip -q install -U flask pyngrok transformers accelerate peft bitsandbytes sentencepiece

In [ ]:
import importlib.metadata as md
print("bitsandbytes:", md.version("bitsandbytes"))

bitsandbytes: 0.49.1


In [ ]:
from google.colab import files
files.upload()

Saving propchk_mistral_lora.zip to propchk_mistral_lora (2).zip
Buffered data was truncated after reaching the output size limit.

In [ ]:
!unzip -o propchk_mistral_lora.zip -d /content
!ls -lh /content/propchk_mistral_lora

Archive:  propchk_mistral_lora.zip
   creating: /content/propchk_mistral_lora/
  inflating: /content/propchk_mistral_lora/adapter_config.json  
  inflating: /content/propchk_mistral_lora/adapter_model.safetensors  
  inflating: /content/propchk_mistral_lora/app.py  
  inflating: /content/propchk_mistral_lora/chat_template.jinja  
  inflating: /content/propchk_mistral_lora/README.md  
  inflating: /content/propchk_mistral_lora/rng_state.pth  
  inflating: /content/propchk_mistral_lora/scaler.pt  
  inflating: /content/propchk_mistral_lora/scheduler.pt  
  inflating: /content/propchk_mistral_lora/special_tokens_map.json  
  inflating: /content/propchk_mistral_lora/tokenizer_config.json  
  inflating: /content/propchk_mistral_lora/trainer_state.json  
  inflating: /content/propchk_mistral_lora/training_args.bin  
total 81M
-rw-rw-rw- 1 root root 1.1K Dec 21 19:56 adapter_config.json
-rw-rw-rw- 1 root root  81M Dec 21 20:00 adapter_model.safetensors
-rw-rw-rw- 1 root root 2.2K Dec 22 13:07

##**Uploading and extracting the fine-tuned LoRA model**

In [ ]:
%%writefile /content/app.py
import re
import torch
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

app = Flask(__name__)

BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"
LORA_PATH  = "/content/propchk_mistral_lora"

# -----------------------------
# Load model (4-bit) + LoRA
# -----------------------------
bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

print("🔹 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

print("🔹 Loading base model (4-bit) on GPU...")
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb,
    device_map="auto"
)
base.config.use_cache = False

print("🔹 Loading LoRA adapter...")
model = PeftModel.from_pretrained(base, LORA_PATH)
model.eval()


# -----------------------------
# Output formatting helper
# -----------------------------
SECTIONS = [
    "Issue type:",
    "Severity:",
    "How to inspect:",
    "Red flags:",
    "Recommendation:",
    "Questions to ask builder:"
]

def _cleanup_spaces(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

def format_response(text: str) -> str:
    """
    Converts single-paragraph response into clean multi-line response.

    Also converts the Questions section into bullets if it contains multiple questions.
    """
    if not text:
        return text

    # Remove accidental prompt remnants
    text = text.replace("### Instruction:", "").replace("### Response:", "").strip()

    # If model already used line breaks, keep them but still clean a bit
    # We'll parse by finding sections.
    extracted = {}

    for i, sec in enumerate(SECTIONS):
        if sec not in text:
            continue

        after = text.split(sec, 1)[1]
        next_pos = len(after)

        # find the earliest next section that appears in 'after'
        for sec2 in SECTIONS[i+1:]:
            p = after.find(sec2)
            if p != -1:
                next_pos = min(next_pos, p)

        extracted[sec] = _cleanup_spaces(after[:next_pos])

    # Fallback: if nothing parsed, just return cleaned text
    if not extracted:
        return text

    # Bulletify questions
    q_key = "Questions to ask builder:"
    if q_key in extracted:
        q_text = extracted[q_key]

        # Split into questions by '?' if many
        parts = [p.strip() for p in re.split(r"\?\s*", q_text) if p.strip()]
        if len(parts) >= 2:
            bullets = []
            for p in parts:
                # add back '?' unless it already ends with one
                if not p.endswith("?"):
                    p = p + "?"
                bullets.append(f"• {p}")
            extracted[q_key] = "\n" + "\n".join(bullets)
        else:
            extracted[q_key] = q_text

    # Build final formatted output
    out_lines = []
    for sec in SECTIONS:
        if sec in extracted:
            value = extracted[sec]
            if sec == q_key and value.startswith("\n"):
                out_lines.append(f"{sec}{value}")
            else:
                out_lines.append(f"{sec} {value}".strip())

    return "\n\n".join(out_lines)


# -----------------------------
# Routes
# -----------------------------
@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "ok"})

@app.route("/generate", methods=["POST"])
def generate():
    data = request.get_json(force=True)
    user_prompt = (data.get("prompt") or "").strip()
    if not user_prompt:
        return jsonify({"error": "Missing prompt"}), 400

    # Strong formatting instruction to help model
    prompt = f"""
### Instruction:
{user_prompt}

Respond strictly in this format:
Issue type:
Severity:
How to inspect:
Red flags:
Recommendation:
Questions to ask builder:

### Response:
""".strip()

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=int(data.get("max_new_tokens", 220)),
            do_sample=True,
            temperature=float(data.get("temperature", 0.4)),
            top_p=float(data.get("top_p", 0.9)),
            repetition_penalty=float(data.get("repetition_penalty", 1.15)),
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(out[0], skip_special_tokens=True)

    # keep only response portion
    if "### Response:" in decoded:
        decoded = decoded.split("### Response:", 1)[1].strip()

    cleaned = format_response(decoded)
    return jsonify({"generated_text": cleaned})


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


Writing /content/app.py


##**Creating the Flask API (app.py)**

####Starting Flask in background (nohup)

In [ ]:
!pkill -f "/content/app.py" || true
!fuser -k 5000/tcp || true
!rm -f /content/flask.log

!nohup python3 /content/app.py > /content/flask.log 2>&1 &
print("✅ Flask started in background")

^C
✅ Flask started in background


####Verifying Flask locally

In [ ]:
import requests
print(requests.get("http://127.0.0.1:5000/health", timeout=5).text)


{"status":"ok"}



####Creating a public URL using Ngrok

In [ ]:
from pyngrok import ngrok
ngrok.kill()

ngrok.set_auth_token("37FwvYI1beNr5RbhUtgdqDFFCrX_56QVVXoikHVvUjMDK4sFu")

tunnel = ngrok.connect(5000, "http")
print("✅ PUBLIC URL:", tunnel.public_url)

✅ PUBLIC URL: https://lesley-readaptive-dialytically.ngrok-free.dev


###**Testing the /generate endpoint (local + public)**

In [ ]:
import requests
print(requests.get("http://127.0.0.1:5000/health").text)

payload = {"prompt": "Bathroom: leakage under wash basin. What to inspect and how to fix?"}
print(requests.post("http://127.0.0.1:5000/generate", json=payload).json())


{"status":"ok"}

{'generated_text': 'Issue type: Leakage under wash basin\n\nSeverity: Major\n\nHow to inspect: Run water and check trap; inspect joints and outlet; retest after tightening.\n\nRed flags: Visible damage, repeated failure, leakage/overheating signs, or unsafe condition.\n\nRecommendation: Tighten/replace fittings; clean trap; confirm outlet alignment; retest.\n\nQuestions to ask builder:\n• Can we re-test after rectification before final handover?\n• Will you rectify this under finishing/warranty scope?\n• Can you share the test report (pressure/earthing/waterproofing) if available?'}


In [ ]:
import requests

API = "https://lesley-readaptive-dialytically.ngrok-free.dev/generate"

payload = {
    "prompt": "Living Room: Flooring hollowness observed. What does it mean and what should I do?"
}

print(requests.post(API, json=payload, timeout=120).json())

{'generated_text': 'Issue type: Flooring hollowness\n\nSeverity: Major if widespread; Minor if isolated\n\nHow to inspect: Tap-test flooring; map hollow tiles; check grout cracks and movement.\n\nRed flags: Visible damage, repeated failure, leakage/overheating signs, or unsafe condition.\n\nRecommendation: Re-fix affected tiles with proper adhesive; regrout; verify level.\n\nQuestions to ask builder:\n• Can we re-test after rectification before final handover?\n• Will you rectify this under finishing/warranty scope?\n• Can you share the test report (pressure/earthing/waterproofing) if available?'}


##**Streamlit**

In [ ]:
!pip -q install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 132.9 MB/s eta 0:00:00


In [ ]:
%%writefile /content/streamlit_app.py
import streamlit as st
import requests

st.set_page_config(page_title="PropChk Inspection Bot", page_icon="🏠", layout="centered")

with st.sidebar:
    st.title("🏠 PropChk Bot")
    st.markdown("""
    **Home Inspection & Handover Assistant**

    Ask about:
    - Defects
    - Inspection checks
    - Severity
    - Rectification
    - Builder questions
    """)

    API_BASE = st.text_input("Flask API URL", value="http://127.0.0.1:5000").strip().rstrip("/")

    st.markdown("---")
    if st.button("🧹 Clear chat"):
        st.session_state.messages = []
        st.rerun()

if "messages" not in st.session_state:
    st.session_state.messages = [{
        "role": "assistant",
        "content": "Hi! 👋 Tell me the **space + issue**.\n\nExample:\n`Bathroom: leakage under wash basin. What to inspect and how to fix?`"
    }]

st.title("💬 PropChk Home Inspection Assistant")
st.caption("Practical inspection guidance for buyers & handover checks")

for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

def call_api(prompt: str) -> str:
    url = f"{API_BASE}/generate"
    payload = {
        "prompt": prompt,
        "max_new_tokens": 220,
        "temperature": 0.4,
        "top_p": 0.9,
        "repetition_penalty": 1.15
    }
    r = requests.post(url, json=payload, timeout=180)
    r.raise_for_status()
    return (r.json().get("generated_text") or "").strip()

user_prompt = st.chat_input("Type your inspection question…")

if user_prompt:
    st.session_state.messages.append({"role": "user", "content": user_prompt})
    with st.chat_message("user"):
        st.markdown(user_prompt)

    with st.chat_message("assistant"):
        with st.spinner("Analyzing inspection issue…"):
            try:
                answer = call_api(user_prompt)
                st.markdown(answer)
                st.session_state.messages.append({"role": "assistant", "content": answer})
            except Exception as e:
                st.error(f"API error: {e}")
                st.info("Make sure Flask is running and `/health` returns OK.")


Writing /content/streamlit_app.py


In [ ]:
!pkill -f streamlit || true
!fuser -k 8501/tcp || true
!nohup streamlit run /content/streamlit_app.py --server.port 8501 --server.address 0.0.0.0 > /content/streamlit.log 2>&1 &
print("✅ Streamlit started in background")


^C
✅ Streamlit started in background


In [ ]:
import time
time.sleep(3)
!tail -n 40 /content/streamlit.log




  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501



In [ ]:
from pyngrok import ngrok
ngrok.kill()

tunnel = ngrok.connect(8501, "http")
print("✅ STREAMLIT URL:", tunnel.public_url)

✅ STREAMLIT URL: https://lesley-readaptive-dialytically.ngrok-free.dev
